In [ ]:
# In a Jupyter Notebook cell

import requests
from IPython.display import display, HTML, Image
import json # For pretty printing errors if needed

# --- Configuration ---
BASE_URL = "https://www.themealdb.com/api/json/v1/1/"

# --- 1. Base API Fetch Function ---
def fetch_api_data(endpoint, params=None):
    """
    Fetches data from TheMealDB API.

    Args:
        endpoint (str): The API endpoint (e.g., "search.php").
        params (dict, optional): A dictionary of query parameters. Defaults to None.

    Returns:
        dict: The JSON response from the API, or an error dictionary.
    """
    try:
        url = f"{BASE_URL}{endpoint}"
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raises an HTTPError for bad responses (4XX or 5XX)
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - Status Code: {response.status_code}")
        try:
            error_content = response.json() # Try to parse JSON error from API
            print("API Error Content:")
            print(json.dumps(error_content, indent=2))
            return {"error": str(http_err), "details": error_content, "status_code": response.status_code}
        except ValueError: # If error response is not JSON
            print("API Error Content (Non-JSON):")
            print(response.text)
            return {"error": str(http_err), "raw_content": response.text, "status_code": response.status_code}
    except requests.exceptions.RequestException as req_err:
        print(f"Request error occurred: {req_err}")
        return {"error": str(req_err)}
    except ValueError as json_err: # If response is not JSON
        print(f"JSON decoding error: {json_err}")
        return {"error": "Failed to decode JSON from response.", "raw_content": response.text if 'response' in locals() else "No response object"}

# --- 2. Specific API Call Functions ---

def search_meal_by_name(meal_name):
    """Searches for meals by name."""
    if not meal_name:
        print("Error: Meal name cannot be empty.")
        return {"error": "Meal name cannot be empty."}
    return fetch_api_data("search.php", params={"s": meal_name})

def list_meals_by_first_letter(letter):
    """Lists meals starting with a specific letter."""
    if not letter or len(letter) != 1:
        print("Error: Please provide a single letter.")
        return {"error": "Invalid letter provided."}
    return fetch_api_data("search.php", params={"f": letter})

def get_random_meal():
    """Gets a single random meal with full details."""
    return fetch_api_data("random.php")

def filter_by_main_ingredient(ingredient_name):
    """Filters meals by a main ingredient."""
    if not ingredient_name:
        print("Error: Ingredient name cannot be empty.")
        return {"error": "Ingredient name cannot be empty."}
    return fetch_api_data("filter.php", params={"i": ingredient_name})

def filter_by_category(category_name):
    """Filters meals by category."""
    if not category_name:
        print("Error: Category name cannot be empty.")
        return {"error": "Category name cannot be empty."}
    return fetch_api_data("filter.php", params={"c": category_name})

def filter_by_area(area_name):
    """Filters meals by area/cuisine."""
    if not area_name:
        print("Error: Area name cannot be empty.")
        return {"error": "Area name cannot be empty."}
    return fetch_api_data("filter.php", params={"a": area_name})


# --- 3. Display Functions for Jupyter Notebook ---

def display_meal_summary_card(meal):
    """Displays a summary card for a meal (typically from search/filter results)."""
    html_output = f"""
    <div style="border: 1px solid #ddd; border-radius: 8px; margin: 10px; padding: 15px; width: 300px; display: inline-block; vertical-align: top; box-shadow: 2px 2px 5px #ccc;">
        <h3 style="margin-top:0;">{meal.get('strMeal', 'N/A')}</h3>
    """
    if meal.get('strMealThumb'):
        html_output += f'<img src="{meal["strMealThumb"]}" alt="{meal.get("strMeal", "Meal image")}" style="width:100%; max-width:280px; border-radius: 5px; margin-bottom:10px;">'
    
    # Filter/Search results don't usually have category/area directly in the meal object
    # but if they did, you could add them here.
    # Example if present:
    # if meal.get('strCategory'):
    #     html_output += f'<p><strong>Category:</strong> {meal["strCategory"]}</p>'
    # if meal.get('strArea'):
    #     html_output += f'<p><strong>Area:</strong> {meal["strArea"]}</p>'
        
    html_output += "</div>"
    display(HTML(html_output))

def display_detailed_meal_card(meal):
    """Displays a detailed card for a single meal (typically from random.php or lookup.php)."""
    if not meal:
        display(HTML("<p>No meal data to display.</p>"))
        return

    html_output = f"""
    <div style="border: 1px solid #ddd; border-radius: 10px; margin: 15px; padding: 20px; max-width: 700px; box-shadow: 3px 3px 8px #bbb;">
        <h2 style="color: #c0392b; margin-top:0;">{meal.get('strMeal', 'N/A')}</h2>
    """
    if meal.get('strMealThumb'):
        html_output += f'<img src="{meal["strMealThumb"]}" alt="{meal.get("strMeal", "Meal image")}" style="width:100%; max-width:400px; border-radius: 8px; margin-bottom:15px; display:block; margin-left:auto; margin-right:auto;">'
    
    if meal.get('strCategory'):
        html_output += f'<p><strong>Category:</strong> <span style="background-color: #f1c40f; padding: 3px 7px; border-radius: 5px;">{meal["strCategory"]}</span></p>'
    if meal.get('strArea'):
        html_output += f'<p><strong>Area:</strong> <span style="background-color: #3498db; color: white; padding: 3px 7px; border-radius: 5px;">{meal["strArea"]}</span></p>'
    
    # Ingredients and Measures
    ingredients_html = "<h4><i class='fas fa-shopping-cart'></i> Ingredients:</h4><ul>"
    has_ingredients = False
    for i in range(1, 21):
        ingredient = meal.get(f'strIngredient{i}')
        measure = meal.get(f'strMeasure{i}')
        if ingredient and ingredient.strip():
            has_ingredients = True
            ingredients_html += f"<li>{ingredient.strip()} - <em>{measure.strip() if measure else ''}</em></li>"
    ingredients_html += "</ul>"
    if has_ingredients:
        html_output += ingredients_html
    
    # Instructions
    if meal.get('strInstructions'):
        instructions = meal['strInstructions'].replace('\r\n', '<br>').replace('\n', '<br>')
        html_output += f"""
        <h4><i class='fas fa-book-open'></i> Instructions:</h4>
        <div style="background-color: #f9f9f9; border-left: 3px solid #c0392b; padding: 10px; margin-top:10px; line-height:1.6;">
            {instructions}
        </div>
        """
        
    # YouTube Link
    if meal.get('strYoutube'):
        html_output += f'<p style="margin-top:15px;"><a href="{meal["strYoutube"]}" target="_blank" style="background-color: #e74c3c; color: white; padding: 10px 15px; text-decoration: none; border-radius: 5px; display:inline-block;">Watch on YouTube <i class="fab fa-youtube"></i></a></p>'
    
    # Source Link
    if meal.get('strSource'):
         html_output += f'<p style="margin-top:10px;"><a href="{meal["strSource"]}" target="_blank" style="background-color: #2ecc71; color: white; padding: 10px 15px; text-decoration: none; border-radius: 5px; display:inline-block;">View Source <i class="fas fa-external-link-alt"></i></a></p>'

    html_output += "</div>"
    
    # Add Font Awesome if you want icons (optional, include this once at the top of notebook or in first display)
    display(HTML('<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css">'))
    display(HTML(html_output))

def display_meal_list(data, title="Meal Results"):
    """
    Displays a list of meals, using summary cards.
    Handles cases where no meals are found or an error occurred.
    """
    display(HTML(f"<h2>{title}</h2>"))
    if not data or data.get("error"):
        error_message = data.get("error", "Unknown error") if data else "No data received"
        details = data.get("details") if data else None
        raw_content = data.get("raw_content") if data else None
        
        html_error = f"<p style='color:red;'><strong>Error:</strong> {error_message}</p>"
        if details:
             html_error += f"<pre>Details: {json.dumps(details, indent=2)}</pre>"
        if raw_content:
             html_error += f"<pre>Raw Content: {raw_content}</pre>"
        display(HTML(html_error))
        return

    meals = data.get("meals")
    if not meals: # API returns {"meals": null} if no results
        display(HTML("<p>No meals found matching your criteria.</p>"))
        return
    
    # Display meals in a flex container for better layout
    html_container_start = '<div style="display: flex; flex-wrap: wrap; justify-content: flex-start;">'
    html_container_end = '</div>'
    
    display(HTML(html_container_start))
    for meal in meals:
        display_meal_summary_card(meal)
    display(HTML(html_container_end))

In [13]:
# Example: Search for meals containing "Arrabiata"
arrabiata_meals = search_meal_by_name("Arrabiata")
display_meal_list(arrabiata_meals, title="Search Results for 'Arrabiata'")

# Example: Search for a non-existent meal
non_existent_meals = search_meal_by_name("XyzNonExistentMeal")
display_meal_list(non_existent_meals, title="Search Results for 'XyzNonExistentMeal'")

In [7]:
# Example: Get a random meal (this will show full details)
random_meal_data = get_random_meal()

if random_meal_data and random_meal_data.get("meals"):
    display(HTML("<h2>Random Meal</h2>"))
    display_detailed_meal_card(random_meal_data["meals"][0]) # random.php returns a list with one meal
elif random_meal_data and random_meal_data.get("error"):
    display_meal_list(random_meal_data, title="Error Fetching Random Meal") # Use display_meal_list to show error
else:
    display(HTML("<p>Could not fetch a random meal.</p>"))

In [12]:
# Example: Filter meals by "Italian" area
italian_meals = filter_by_area("Uruguayan")
display_meal_list(italian_meals, title="Uruguayan Cuisine")